In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
from sklearn.utils import shuffle
import jieba
import re
import time
import collections
import os

In [2]:
question = open("question.csv",encoding='utf-8')

answer = open("answer.csv",encoding='utf-8')

all_train_id = pd.read_table("train_candidates.txt",sep=',',dtype=str,nrows = None)

In [3]:
# # txt去重，太慢了
# train_set = []
# for line in train_id:
#     que_to_ans = line.strip('\n').split(",")[:2]
#     if que_to_ans not in train_set:
#         print(que_to_ans)
#         train_set.append(que_to_ans)
# train_set

del all_train_id["neg_ans_id"]
all_train_id.drop_duplicates(inplace=True)
all_train_id.reset_index(drop=True, inplace=True)
train_id = all_train_id[:2000]

In [4]:
# 简单处理一下question/answer每句话
def sentence_precessing(text):
    
    text = text.lower().strip()
    
    text = re.sub(r'["cm"]+', "厘米", text)
    text = re.sub(r"([。])", r".", text)
    text = re.sub(r"[a-zA-Z]", "",text)
    text = re.sub(r"([？?!！，¿])", r",", text)
    text = re.sub(r"([‘’“”\"\"''])", r"", text)
    text = re.sub(r"[-()（）\#/@;:<>{}`+=~|]", "", text)
    text = re.sub(r'[" "]+', "", text)
    
    return text

In [5]:
question_dict = {}
question.readline()
for line in question:
    _line = line.strip('\n').split(',')
    question_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[1])))

answer_dict = {}
answer.readline()
for line in answer:
    _line = line.strip('\n').split(',')
    answer_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[2])))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\leo\AppData\Local\Temp\jieba.cache
Loading model cost 0.874 seconds.
Prefix dict has been built succesfully.


In [6]:
train_questions = []
train_answers = []
for question,answer in zip(train_id["question_id"],train_id["pos_ans_id"]):
    train_questions.append(question_dict[question])
    train_answers.append(answer_dict[answer])

In [7]:
def build_dataset(words, n_words, atleast=3):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [8]:
concat_from = ' '.join(train_questions).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 4757
Most common words [(',', 5694), ('了', 1311), ('的', 1259), ('.', 1064), ('我', 962), ('是', 783)]
Sample data [170, 41, 25, 1490, 42, 141, 11, 4, 215, 14] ['不是', '说', '做', '超对', '宝宝', '不好', '吗', ',', '那', '怀孕']
filtered vocab size: 2292
% of vocab used: 48.18%


In [9]:
concat_to = ' '.join(train_answers).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 10294
Most common words [(',', 10837), ('的', 6329), ('.', 4774), ('、', 2278), ('是', 2057), ('治疗', 1523)]
Sample data [254, 197, 0, 298, 17, 8, 274, 5, 352, 26] ['超', '属于', 'PAD', '经常', '检查', '是', '不好', '的', '而且', '也']
filtered vocab size: 3858
% of vocab used: 37.480000000000004%


In [10]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [11]:
for i in range(len(train_answers)):
    train_answers[i] += ' EOS'

In [12]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [13]:
X = str_idx(train_questions, dictionary_from)
Y = str_idx(train_answers, dictionary_to)
# X_test = str_idx(question_test, dictionary_from)
# Y_test = str_idx(answer_test, dictionary_from)

In [37]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [15]:
maxlen_question = max([len(x) for x in X])
maxlen_answer = max([len(y) for y in Y])

In [16]:
emb_size = 256
n_hidden = 256
n_layers = 4
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [17]:
def encoder_block(inp, n_hidden, filter_size):
    inp = tf.expand_dims(inp, 2)
    inp = tf.pad(inp, [[0, 0], [(filter_size[0]-1)//2, (filter_size[0]-1)//2], [0, 0], [0, 0]])
    conv = tf.layers.conv2d(inp, n_hidden, filter_size, padding="VALID", activation=None)
    conv = tf.squeeze(conv, 2)
    return conv

def glu(x):
    return tf.multiply(x[:, :, :tf.shape(x)[2]//2], tf.sigmoid(x[:, :, tf.shape(x)[2]//2:]))

def layer(inp, conv_block, kernel_width, n_hidden, residual=None):
    z = conv_block(inp, n_hidden, (kernel_width, 1))
    return glu(z) + (residual if residual is not None else 0)

In [38]:
class Chatbot:
    def __init__(self):

        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])

        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        encoder_embedding = tf.Variable(tf.random_uniform([len(dictionary_from), emb_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([len(dictionary_to), emb_size], -1, 1))
        
        encoder_embedded = tf.nn.embedding_lookup(encoder_embedding, self.X)
        
        e = tf.identity(encoder_embedded)
        for i in range(n_layers):
            z = layer(encoder_embedded, encoder_block, 3, n_hidden * 2, encoder_embedded)
            encoder_embedded = z
        
        encoder_output, output_memory = z, z + e
        
        vocab_proj = tf.layers.Dense(len(dictionary_to))
        init_state = tf.reduce_mean(output_memory,axis=1)
        cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
        helper = tf.contrib.seq2seq.TrainingHelper(
            inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
            sequence_length = tf.to_int32(self.Y_seq_len))
        encoder_state = tf.nn.rnn_cell.LSTMStateTuple(c=init_state, h=init_state)
        decoder = tf.contrib.seq2seq.BasicDecoder(cell = cell,
                                                  helper = helper,
                                                  initial_state = encoder_state,
                                                  output_layer = vocab_proj)
        decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder = decoder,
                                                                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        
        helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding = decoder_embedding,
                                                          start_tokens = tf.tile(
                                                              tf.constant([GO], 
                                                                          dtype=tf.int32), 
                                                              [tf.shape(init_state)[0]]),
                                                          end_token = EOS)
        decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = cell,
            helper = helper,
            initial_state = encoder_state,
            output_layer = vocab_proj)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = decoder,
            maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
        self.training_logits = decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.sample_id
        self.logits = decoder_output.sample_id
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [39]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot()
sess.run(tf.global_variables_initializer())

C:\Users\leo\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [41]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, len(train_questions), batch_size):
        index = min(k+batch_size, len(train_questions))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD)
        predicted, accuracy,loss, _ = sess.run([model.predicting_ids,model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
        print('batch: %d, avg loss: %f, avg accuracy: %f'%(k+1, loss, accuracy))
        
    total_loss /= (len(train_questions) / batch_size)
    total_accuracy /= (len(train_questions) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

batch: 1, avg loss: 8.138039, avg accuracy: 0.078664
batch: 129, avg loss: 8.032100, avg accuracy: 0.089931
batch: 257, avg loss: 7.886002, avg accuracy: 0.080641
batch: 385, avg loss: 7.637905, avg accuracy: 0.096117


KeyboardInterrupt: 

In [ ]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

In [ ]:
batch_x, seq_x = pad_sentence_batch(X_test[:batch_size], PAD)
batch_y, seq_y = pad_sentence_batch(Y_test[:batch_size], PAD)
predicted = sess.run(model.predicting_ids, feed_dict={model.X:batch_x})

for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')